<a href="https://colab.research.google.com/github/Kepners/ChopOnions/blob/main/Trndzo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
# Block 1: Install Required Libraries
!pip install praw openai pytrends feedparser requests cachetools IPython prettytable textblob


In [98]:
# Block 2: Mount Google Drive and Load Environment Variables
from google.colab import drive
import os
from dotenv import load_dotenv

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your .env file in Google Drive
dotenv_path = '/content/drive/MyDrive/Secrets/.env'

# Load the environment variables from the .env file
load_dotenv(dotenv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [99]:
# Block 3: Configure Logging
import logging

# Configure Logging to write to a log file and suppress console output
log_file = 'app.log'
logging.basicConfig(
    level=logging.ERROR,  # Only log ERROR and CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(open(os.devnull, 'w'))  # Suppress console output
    ]
)


In [100]:
# Block 4: Download NLTK Data Silently
import nltk
import sys
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('brown')
    nltk.download('wordnet')
    nltk.download('movie_reviews')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [101]:
# Block 5: Create Utils Directory
import os

utils_dir = 'utils'
if not os.path.exists(utils_dir):
    os.makedirs(utils_dir)


In [102]:
# Block 6: Create data_processing.py
data_processing_code = """
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textblob import TextBlob

def extract_keywords(text, max_keywords=10):
    \"""
    Extracts up to `max_keywords` nouns from the input `text`.
    \"""
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
    # Get part-of-speech tags
    tagged_words = pos_tag(filtered_words)
    # Keep nouns and proper nouns
    keywords = [word for word, pos in tagged_words if pos.startswith('NN')]
    # Limit the number of keywords
    return keywords[:max_keywords]

def analyze_sentiment(text):
    \"""
    Analyzes the sentiment of the given text and returns it.
    \"""
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.1:
        return 'Positive'
    elif polarity < -0.1:
        return 'Negative'
    else:
        return 'Neutral'
"""
with open(os.path.join('utils', 'data_processing.py'), 'w') as file:
    file.write(data_processing_code)


In [103]:
# Block 7: Initialize API Clients and Configure Logging
import praw
import openai
import logging
import warnings
from utils.data_processing import extract_keywords, analyze_sentiment
from cachetools import TTLCache, cached
from pytrends.request import TrendReq
import feedparser
import requests
import time
import os

# Suppress PRAW warnings about asynchronous environments
warnings.filterwarnings("ignore", category=UserWarning, module='praw')
logging.getLogger('praw').setLevel(logging.CRITICAL)

# Initialize Reddit API using PRAW
try:
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT', 'script:TrendingTopicsScript:1.0 (by u/yourusername)')
    )
except Exception as e:
    logging.error(f"Error initializing Reddit API: {e}")

# Initialize OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')

# Initialize Pexels API key
PEXELS_API_KEY = os.getenv('PEXELS_API_KEY')

# Initialize Shutterstock Access Token
SHUTTERSTOCK_ACCESS_TOKEN = os.getenv('SHUTTERSTOCK_ACCESS_TOKEN')

# Define a list of countries with their codes
available_countries = {
    'United States': 'US',
    'India': 'IN',
    'Canada': 'CA',
    'United Kingdom': 'GB',
    'Australia': 'AU',
    'Germany': 'DE',
    'France': 'FR',
    'Brazil': 'BR',
    'Mexico': 'MX',
    'Japan': 'JP',
    'Russia': 'RU',
    'South Korea': 'KR',
    'Italy': 'IT',
    'Spain': 'ES',
    'Netherlands': 'NL',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'Austria': 'AT',
    'Belgium': 'BE',
    'New Zealand': 'NZ'
}

# Mapping from country codes to PyTrends 'pn' parameter for trending searches
# For realtime_trending_searches, the acceptable 'pn' values are specific
country_code_to_pn_realtime = {
    'US': 'US',
    'IN': 'IN',
    'GB': 'GB',
    'CA': 'CA',
    'AU': 'AU',
    'DE': 'DE',
    'FR': 'FR',
    'BR': 'BR',
    'MX': 'MX',
    'JP': 'JP',
    'RU': 'RU',
    'KR': 'KR'  # South Korea
}

# For trending_searches, the acceptable 'pn' values are:
country_code_to_pn_daily = {
    'united_states': 'united_states',
    'india': 'india',
    'united_kingdom': 'united_kingdom',
    'canada': 'canada',
    'australia': 'australia',
    'germany': 'germany',
    'france': 'france',
    'brazil': 'brazil'
    # Note: Limited countries are available for trending_searches
}


ImportError: cannot import name 'analyze_sentiment' from 'utils.data_processing' (/content/utils/data_processing.py)

In [104]:
# Block 8: Define Supporting Functions
def fetch_trending_topics_pytrends(selected_country_code, time_range):
    try:
        pytrends = TrendReq(hl='en-US', tz=360)
        if time_range == '4h':
            # Realtime trending searches
            pn = country_code_to_pn_realtime.get(selected_country_code)
            if not pn:
                logging.error(f"Realtime trending searches not available for the selected country.")
                return []
            df = pytrends.realtime_trending_searches(pn=pn)
        else:
            # Daily trending searches
            pn = country_code_to_pn_daily.get(selected_country_code.lower())
            if not pn:
                logging.error(f"Daily trending searches not available for the selected country.")
                return []
            df = pytrends.trending_searches(pn=pn)
        if df.empty:
            return []
        trending_topics = []
        for index, row in df.iterrows():
            title = row[0]
            # Generate a brief reason why the topic is trending
            reason = generate_trend_reason(title)
            trending_topics.append({
                'title': title,
                'description': reason,  # Brief reason why it's trending
                'sv': 'High',       # Placeholder for Search Volume
                'change': 'Up',     # Placeholder for Change
                'started': 'Recently'   # Placeholder
            })
        return trending_topics[:10]  # Limit to top 10
    except Exception as e:
        logging.error(f"Error fetching Google Trends data via PyTrends: {e}")
        return []

def generate_trend_reason(topic):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that provides brief reasons why a topic is trending. Your responses should be concise, in brackets, and two to four words long."
                },
                {
                    "role": "user",
                    "content": f"Why is '{topic}' trending? Provide a brief reason in brackets, two to four words long."
                }
            ],
            max_tokens=20,
            temperature=0.5,
        )
        reason = response['choices'][0]['message']['content'].strip()
        return reason
    except Exception as e:
        logging.error(f"Error generating trend reason: {e}")
        return "[Trending Topic]"

def generate_summary(content):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a concise summarizer. Provide a clear and brief two-sentence summary of the following content."
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            max_tokens=80,
            temperature=0.5,
        )
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        return "No summary available."

def search_subreddits_for_topic(topic, limit=5):
    """
    Searches for subreddits related to the topic.
    """
    try:
        related_subreddits = []
        subreddits_found = reddit.subreddits.search_by_name(query=topic, exact=False, include_nsfw=False)
        for subreddit in subreddits_found:
            # Fetch subreddit details
            try:
                subreddit_details = reddit.subreddit(subreddit.display_name)
                # Check if subreddit is accessible
                if subreddit_details.over18 or subreddit_details.subreddit_type in ['private', 'restricted']:
                    continue
                related_subreddits.append({
                    'name': subreddit.display_name,
                    'title': subreddit.title,
                    'description': subreddit.public_description or ''
                })
                if len(related_subreddits) >= limit:
                    break
            except Exception as inner_e:
                logging.error(f"Error accessing subreddit '{subreddit.display_name}': {inner_e}")
                continue
        return related_subreddits
    except Exception as e:
        logging.error(f"Error searching for subreddits: {e}")
        return []

def suggest_subreddits_via_openai(topic):
    """
    Uses OpenAI to suggest relevant subreddit names based on the topic.
    """
    try:
        prompt = f"List 5 subreddit names that are relevant to the topic '{topic}'. Only provide the subreddit names without any additional text."
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0.5,
            n=1,
        )
        suggestions = response['choices'][0]['message']['content'].strip().split('\n')
        subreddits = [s.strip().lstrip('/r/') for s in suggestions if s.strip()]
        return subreddits
    except Exception as e:
        logging.error(f"Error suggesting subreddits via OpenAI: {e}")
        return []

def fetch_related_news(topic, limit=5):
    """
    Fetches related news articles using Google News RSS feed.
    """
    try:
        feed_url = f"https://news.google.com/rss/search?q={requests.utils.quote(topic)}&hl=en-US&gl=US&ceid=US:en"
        feed = feedparser.parse(feed_url)
        related_stories = []
        for entry in feed.entries[:limit]:
            title = entry.title
            summary = entry.summary if 'summary' in entry else ''
            link = entry.link
            related_stories.append({'title': title, 'summary': summary, 'url': link})
        return related_stories
    except Exception as e:
        logging.error(f"Error fetching related news: {e}")
        return []

@cached(TTLCache(maxsize=1000, ttl=86400))
def check_stock_media_availability(topic):
    # Extract keywords from the topic
    keywords = extract_keywords(topic)
    if not keywords:
        logging.error(f"No valid keywords extracted for topic: {topic}")
        return 0
    # Join keywords into a query string
    query = ' '.join(keywords)[:100]
    total_results = 0
    # Check Pexels
    total_results += check_pexels_media(query)
    # Check Shutterstock
    total_results += check_shutterstock_media(query)
    return total_results

def check_pexels_media(query):
    headers = {
        'Authorization': PEXELS_API_KEY,
        'User-Agent': 'TrendingTopicsScript'
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1
    }
    try:
        response = requests.get('https://api.pexels.com/v1/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('total_results', 0)
        else:
            logging.error(f"Pexels API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Pexels for '{query}': {e}")
        return 0

def check_shutterstock_media(query):
    headers = {
        'Authorization': f'Bearer {SHUTTERSTOCK_ACCESS_TOKEN}',
        'User-Agent': 'TrendingTopicsScript'
    }
    params = {
        'query': query,
        'per_page': 1,
        'page': 1,
        'view': 'minimal'
    }
    try:
        response = requests.get('https://api.shutterstock.com/v2/images/search', headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get('total_count', 0)
        else:
            logging.error(f"Shutterstock API error {response.status_code}: {response.text}")
            return 0
    except requests.exceptions.RequestException as e:
        logging.error(f"Error checking Shutterstock for '{query}': {e}")
        return 0

def generate_script_for_topic(topic, content_summary, style="Normal Script"):
    try:
        style_prompts = {
            "Flashy Script": "Create a flashy, high-energy script with quick cuts, bold visuals, and impactful statements.",
            "Expressive Script": "Compose an expressive script that evokes emotions, using descriptive language and a storytelling approach.",
            "Normal Script": "Write a straightforward script that clearly presents the information in an engaging manner."
        }
        style_prompt = style_prompts.get(style, style_prompts["Normal Script"])
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a creative scriptwriter for short-form videos. "
                        "Produce engaging, dynamic scripts that are visually compelling and can be represented "
                        "using stock images and videos. The script should be suitable for a 30-second video, "
                        "have a captivating introduction, a clear narrative flow, and a strong conclusion or call-to-action."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"{style_prompt}\n"
                        f"Using the following summary, write a script suitable for a 30-second video:\n\n"
                        f"Summary: {content_summary}\n\n"
                        "Ensure the script includes vivid descriptions and is structured with a beginning, middle, and end. "
                        "Use language that resonates with the target audience, and make sure it's adaptable with widely available stock images and videos."
                    )
                }
            ],
            max_tokens=800,
            temperature=0.7,
        )
        script_content = response['choices'][0]['message']['content'].strip()
        return script_content
    except Exception as e:
        logging.error(f"Error generating script: {e}")
        return "No script available."


In [105]:
# Block 9: Define the Main Function
def main():
    from IPython.display import display, Markdown, HTML
    from prettytable import PrettyTable

    # Step 1: User selects the country for Google Trends
    print("Enter the country for Google Trends data (e.g., United States):")
    country_input = input("Country: ").strip()
    matching_countries = [country for country in available_countries if country_input.lower() in country.lower()]
    if not matching_countries:
        print("No matching countries found. Exiting.")
        return
    if len(matching_countries) == 1:
        selected_country = matching_countries[0]
    else:
        print("Multiple countries found:")
        for idx, country in enumerate(matching_countries, start=1):
            print(f"{idx}. {country}")
        try:
            country_selection = int(input("Enter the number of the country you're interested in: "))
            if 1 <= country_selection <= len(matching_countries):
                selected_country = matching_countries[country_selection - 1]
            else:
                logging.error("Invalid selection. Exiting.")
                return
        except ValueError:
            logging.error("Invalid input. Please enter a number. Exiting.")
            return

    selected_country_code = available_countries[selected_country]
    print(f"You selected: {selected_country}")

    # Step 1b: User selects the time range for Google Trends
    print("\nSelect the time range for trending topics:")
    print("1. Last 4 hours (Realtime)")
    print("2. Last 24 hours (Daily)")
    time_range_selection = input("Enter the number of the time range you're interested in: ")
    time_range_mapping = {'1': '4h', '2': '24h'}
    time_range = time_range_mapping.get(time_range_selection, '24h')

    # List locations with daily results available
    if time_range == '24h':
        print("\nLocations with daily trending data available:")
        for loc in country_code_to_pn_daily.keys():
            print(f"- {loc.capitalize()}")

    # Step 2: Fetch trending topics for the selected country and time range
    google_trends_topics = fetch_trending_topics_pytrends(selected_country_code, time_range)

    if not google_trends_topics:
        logging.error("No trending topics found using PyTrends.")
        print("No trending topics found for the selected country and time range.")
        return

    # Display the trending topics with additional information in a table
    print(f"\nCurrent Trending Topics in {selected_country}:\n")
    table = PrettyTable()
    table.field_names = ["No.", "Topic", "Reason"]
    for idx, topic in enumerate(google_trends_topics, start=1):
        title = topic['title']
        reason = topic['description'] if topic['description'] else "[Trending Topic]"
        table.add_row([idx, title, reason])
    print(table)

    # Allow the user to select a topic
    try:
        selected_idx = int(input("Enter the number of the topic you're interested in: "))
        if 1 <= selected_idx <= len(google_trends_topics):
            selected_topic = google_trends_topics[selected_idx - 1]['title']
            print(f"\nYou selected: {selected_topic}")
        else:
            logging.error("Invalid selection. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter a number. Exiting.")
        return

    # Step 3: Fetch related news articles
    related_stories = fetch_related_news(selected_topic, limit=5)
    if related_stories:
        print("\nRelated News Articles:")
        for idx, story in enumerate(related_stories, start=1):
            display(Markdown(f"**{idx}. {story['title']}**"))
    else:
        print("\nNo related news articles found.")

    # Step 4: Search for subreddits related to the topic
    related_subreddits = search_subreddits_for_topic(selected_topic, limit=5)
    if related_subreddits:
        print("\nRelated Subreddits:")
        for idx, sub in enumerate(related_subreddits, start=1):
            display(Markdown(f"**{idx}. {sub['name']}** - *{sub['title']}*"))
    else:
        print("\nNo related subreddits found.")

    # Fetch and display top posts from subreddits
    all_posts = []
    if related_subreddits:
        print("\nTop Reddit Posts from Related Subreddits:")
        for sub in related_subreddits:
            subreddit_name = sub['name']
            try:
                subreddit = reddit.subreddit(subreddit_name)
                top_posts = list(subreddit.hot(limit=5))
                for post in top_posts:
                    if post.over_18:
                        continue
                    if post.is_self and len(post.selftext) < 20:
                        continue  # Skip short self-posts
                    all_posts.append({'subreddit': subreddit_name, 'title': post.title, 'score': post.score, 'url': post.url})
            except praw.exceptions.APIException as e:
                logging.error(f"APIException when accessing subreddit '{subreddit_name}': {e}")
            except Exception as e:
                logging.error(f"Error fetching posts from subreddit '{subreddit_name}': {e}")

        if all_posts:
            # Display posts in a table with formatting
            table = PrettyTable()
            table.field_names = ["No.", "Subreddit", "Title", "Upvotes"]
            for idx, post in enumerate(all_posts, start=1):
                table.add_row([idx, post['subreddit'], post['title'], post['score']])
            print(table)
        else:
            print("\nNo suitable Reddit posts found.")
    else:
        print("\nNo Reddit posts to display.")

    # Combine news articles and Reddit posts
    combined_content = []
    for story in related_stories:
        combined_content.append({'type': 'news', 'title': story['title'], 'url': story['url'], 'summary': story.get('summary', '')})
    for post in all_posts:
        combined_content.append({'type': 'reddit', 'title': post['title'], 'url': post['url'], 'score': post['score'], 'summary': ''})

    if not combined_content:
        logging.error("No content available for script generation. Exiting.")
        return

    # Allow user to select content for script generation
    print("\nAvailable Content for Script Generation:")
    table = PrettyTable()
    table.field_names = ["No.", "Type", "Title", "Upvotes"]
    for idx, content in enumerate(combined_content, start=1):
        content_type = "News Article" if content['type'] == 'news' else "Reddit Post"
        upvotes = content.get('score', '')
        table.add_row([idx, content_type, content['title'], upvotes])
    print(table)

    selected_content_input = input("\nEnter the numbers of the items you want to generate scripts for, separated by commas (e.g., 1,3,5): ")
    try:
        selected_indices = [int(i.strip()) - 1 for i in selected_content_input.split(',') if i.strip().isdigit()]
        selected_items = [combined_content[i] for i in selected_indices if 0 <= i < len(combined_content)]
        if not selected_items:
            logging.error("No valid items selected. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter numbers separated by commas. Exiting.")
        return

    # Fetch content and generate summaries for selected items
    print(f"\nFetching content and generating summaries for selected items...")
    for item in selected_items:
        title = item['title']
        if item['type'] == 'reddit':
            # Analyze sentiment and include upvotes
            sentiment = analyze_sentiment(title)
            item['sentiment'] = sentiment
            item['summary'] = generate_fan_sentiment_summary(title, sentiment)
        else:
            # For news articles, use the summary if available
            if not item['summary']:
                item['summary'] = generate_summary(title)

    # Decide Whether to Check Stock Media Availability
    check_media = input("\nDo you want to check for stock media availability for the selected items? (yes/no): ").strip().lower()
    if check_media not in ['yes', 'y']:
        print("\nSkipping stock media availability check.")
        # Proceed to generate scripts without checking
        generate_scripts_from_items(selected_items)
        print("\nScript execution completed.")
        return

    # Continue with stock media availability check
    print("\nChecking stock media availability for the selected items...")
    request_count = 0
    scripts_generated = 0
    for item in selected_items:
        title = item['title']
        summary = item.get('summary', "No summary available.")
        if request_count >= 190:
            logging.error("Approaching API rate limits. Waiting for 60 minutes before continuing...")
            time.sleep(3600)  # Wait for an hour
            request_count = 0
        total_media = check_stock_media_availability(title)
        request_count += 1
        if total_media >= 10:
            # Display summary before generating script
            print(f"\nSummary: {summary}")
            # Generate script for this topic
            generate_script_with_style(title, summary)
            scripts_generated += 1
        else:
            print(f"\nSummary: {summary}")
            print(f"Not enough stock media for topic: {title}\n====\n")
    if scripts_generated == 0:
        logging.error("No topics with sufficient stock media were found.")
    else:
        logging.info(f"Generated {scripts_generated} script(s) based on available stock media.")
    print("\nScript execution completed.")

def generate_scripts_from_items(selected_items):
    for item in selected_items:
        title = item['title']
        summary = item.get('summary', "No summary available.")
        print(f"\nSummary: {summary}")
        generate_script_with_style(title, summary)
        print("====\n")

def generate_script_with_style(topic, summary):
    print("\nSelect a script style:")
    print("1. Flashy Script")
    print("2. Expressive Script")
    print("3. Normal Script")
    style_choice = input("Enter the number of the script style you prefer: ")
    style_mapping = {'1': 'Flashy Script', '2': 'Expressive Script', '3': 'Normal Script'}
    style = style_mapping.get(style_choice, 'Normal Script')
    script = generate_script_for_topic(topic, summary, style=style)
    # Display the script with enhanced formatting
    display(Markdown(f"### Generated {style} for '{topic}':\n\n{script}"))

def generate_fan_sentiment_summary(title, sentiment):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that summarizes fan sentiments from online discussions into a brief overview."
                },
                {
                    "role": "user",
                    "content": f"The following Reddit post has a {sentiment} sentiment:\n\n'{title}'\n\nSummarize this sentiment for inclusion in a video script."
                }
            ],
            max_tokens=80,
            temperature=0.5,
        )
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating fan sentiment summary: {e}")
        return "No summary available."


In [83]:
# Block 9: Run the Main Function
if __name__ == "__main__":
    main()


Select a country for Google Trends data:
1. United States
2. India
3. Canada
4. United Kingdom
5. Australia
6. Germany
7. France
8. Brazil
9. Mexico
10. Japan
11. Russia
12. South Korea
13. Italy
14. Spain
15. Netherlands
16. Sweden
17. Switzerland
18. Austria
19. Belgium
20. New Zealand
Enter the number of the country you're interested in: 1
You selected: United States

Select the time range for trending topics:
1. Last 4 hours (Realtime)
2. Last 24 hours (Daily)
Enter the number of the time range you're interested in: 2

Current Trending Topics in United States:


**1. Penn State football** - *[Big game win]* [SV: High | Change: Up | Started: Recently]

**2. Ohio State vs Penn State** - *[College football rivalry]* [SV: High | Change: Up | Started: Recently]

**3. Georgia football** - *[Big win against Clemson]* [SV: High | Change: Up | Started: Recently]

**4. Florida vs Georgia** - *College football rivalry. [Big game upcoming.]* [SV: High | Change: Up | Started: Recently]

...

**26. Looking for Fall 2025 Housing** *(Type: Reddit Post)*
URL: https://www.reddit.com/r/PennStateHousing/comments/1ge91po/looking_for_fall_2025_housing/

**27. Way to know the length of on campus housing waitlists?** *(Type: Reddit Post)*
URL: https://www.reddit.com/r/PennStateHousing/comments/1g50ei0/way_to_know_the_length_of_on_campus_housing/

**28. DISCOUNTED rate at the VIEW at State College!!!** *(Type: Reddit Post)*
URL: https://www.reddit.com/r/PennStateHousing/comments/1g4pf0i/discounted_rate_at_the_view_at_state_college/

**29. Looking for a sublet for the summer of 2025** *(Type: Reddit Post)*
URL: https://www.reddit.com/r/PennStateHousing/comments/1g4ola6/looking_for_a_sublet_for_the_summer_of_2025/


Enter the numbers of the items you want to generate scripts for, separated by commas (e.g., 1,3,5): 11

Fetching content and generating summaries for selected items...

Do you want to check for stock media availability for the selected items? (yes/no): no

Skipping stock media availability check.

Summary: The speaker expresses a strong preference to see Penn State lose indefinitely rather than witness a negative interaction between Kelce and a fan. They condemn the fan's behavior as utterly disgusting and urge others to distance themselves from such individuals.

Select a script style:
1. Flashy Script
2. Expressive Script
3. Normal Script
Enter the number of the script style you prefer: 1

Generated Flashy Script for 'Kelce incident':
[Opening shot of a football field under bright stadium lights]

Narrator: "In the heart of the game, a moment that shook the gridiron."

[Quick cuts of intense football tackles and catches]

Narrator: "The Kelce incident - where passion met precision."